In [1]:
import os
import folium
import pandas as pd
from time import sleep as slp
from selenium import webdriver
import glob
from selenium.webdriver.chrome.options import Options
import os

In [2]:
os.getcwd()

# 前準備

In [3]:
data = pd.read_csv("./gps.csv",encoding="shift_jis")

In [4]:
data.head()

,人,日時,緯度,経度
0,A,2019/2/22 12:00,35.619700,139.728553
1,A,2019/2/22 12:01,35.626446,139.723444
2,A,2019/2/22 12:02,35.633998,139.715828
3,A,2019/2/22 12:03,35.646690,139.710106
4,A,2019/2/22 12:04,35.658517,139.701334


In [5]:
data[data["人"] == "B"].head()

,人,日時,緯度,経度
30,B,2019/2/22 12:00,35.681382,139.766084
31,B,2019/2/22 12:30,35.630152,139.740440
32,B,2019/2/22 13:00,35.507456,139.617585
33,B,2019/2/22 13:30,35.256420,139.154904
34,B,2019/2/22 14:00,35.103217,139.077760


In [6]:
person_list = data["人"].unique()

In [7]:
color_list=['red','blue','green','purple','orange','darkred','lightred','beige','darkblue','darkgreen','cadetblue','darkpurple','white','pink','lightblue','lightgreen','gray','black','lightgray']

# jupyter notebookにそれぞれの人の動きを表示する。

In [8]:
# 地図オブジェクトを作成
m = folium.Map(tiles='OpenStreetMap')

for idx,person in enumerate(person_list):
    # 一人分のデータだけをdata_tempに格納する
    data_temp = data[data["人"] == person]
    
    # data_tempの順番を日時で昇順ソート
    data_temp = data_temp.sort_values('日時', ascending=True)
    
    # data_tempの緯度経度だけをデータフレームに残す
    data_temp_lat_lon = data_temp[["緯度","経度"]]
    
    # 緯度経度を配列に格納
    locs = data_temp_lat_lon.values
    
    # 色を指定
    line_color = color_list[idx%len(color_list)]
    
    # 地図に線を追加する。緯度経度の配列をそのまま線として使う
    folium.PolyLine(locs,color=line_color,popup=person).add_to(m)
    
# 地図の表示範囲を緯度経度の最低最大とする
m.fit_bounds([[data["緯度"].min(),data["経度"].min()], [data["緯度"].max(),data["経度"].max()]])

# 地図を表示する
m

# 地図を人毎に作成し、htmlで保存し、画像化する

In [9]:
for person in person_list:
    # 一人分のデータだけをdata_tempに格納する
    data_temp = data[data["人"] == person]
    
    # data_tempの順番を日時で昇順ソート
    data_temp = data_temp.sort_values('日時', ascending=True)
    
    # data_tempの緯度経度だけをデータフレームに残す
    data_temp_lat_lon = data_temp[["緯度","経度"]]
    
    # 地図オブジェクトを生成
    m = folium.Map(tiles='OpenStreetMap')
    
    # 緯度経度を配列に格納
    locs = data_temp_lat_lon.values
    
    # 地図に線を追加する。緯度経度の配列をそのまま線として使う
    folium.PolyLine(locs).add_to(m)
    
    # 地図の表示範囲を緯度経度の最低最大とする
    m.fit_bounds([[data_temp["緯度"].min(),data_temp["経度"].min()], [data_temp["緯度"].max(),data_temp["経度"].max()]])
    
    # htmlに保存する
    # ./html/xx.htmlに保存される
    m.save(outfile="./html/"+person+".html")

    # seleniumでブラウザを開く
    browser = webdriver.Chrome()
    
    # 画面を最大化
    browser.maximize_window()
    
    # URLを指定(ローカルファイル)
    cd = os.getcwd().replace('\\','/')
    tmpurl="file://"+cd+"./html/" + person + ".html"
    
    # URLを開く
    browser.get(tmpurl)
    
    # 一応3秒寝かす
    slp(3)
    
    # スクリーンショットを取る
    browser.save_screenshot("./png/"+person+".png")
    
    # ブラウザを閉じる
    browser.quit()